In [ ]:
from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline 
import time, math
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder

window_size=100
num_signal=16

tf.random.set_seed(777)  

xy = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)
print(xy.shape)
xy_te = np.loadtxt('Input_data_path', delimiter=',', dtype=np.float32)
print(xy_te.shape)

xy_data=xy.reshape(-1,window_size,21)
xy_te_data=xy_te.reshape(-1,window_size,21)

np.random.shuffle(xy_data)
np.random.shuffle(xy_te_data)

xy_data=xy.reshape(-1,21)
xy_te_data=xy_te.reshape(-1,21)

y_class=(xy_data[:,-1].reshape(-1,1))
enc = OneHotEncoder(categories='auto')
enc.fit(y_class)
y_class_onehot = enc.transform(y_class).toarray()

y_class_te=(xy_te_data[:,-1].reshape(-1,1))
enc_te = OneHotEncoder(categories='auto')
enc_te.fit(y_class_te)
y_class_onehot_te = enc_te.transform(y_class_te).toarray()


x_data_training = xy_data[:, :-5]
print(x_data_training.shape)
y_data_training = y_class_onehot[:, :]
print(y_data_training.shape)
x_data_test = xy_te_data[:, :-5]
print(x_data_test.shape)
y_data_test = y_class_onehot_te[:, :]
print(y_data_test.shape)

x_data_training=x_data_training.reshape(-1,window_size,num_signal,1)
y_data_training=y_data_training.reshape(-1,window_size,y_class_onehot.shape[1])
x_data_test=x_data_test.reshape(-1,window_size,num_signal,1)
y_data_test=y_data_test.reshape(-1,window_size,y_class_onehot_te.shape[1])
y_data_training=np.mean(y_data_training,axis=1)
y_data_test=np.mean(y_data_test,axis=1)

from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import KFold
from tensorflow import keras

seq_length=100
data_dim=16
num_classes=3
# num_classes=10
# num_classes=9
output_dim=num_classes
window_size = 100
num_signal = 16

model = models.Sequential()
model.add(layers.Conv2D(64, (16, 16), strides=(1,1), padding='same', activation='relu', 
                        kernel_initializer='glorot_uniform', input_shape=(window_size, num_signal, 1)))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.Conv2D(128, (16, 16), strides=(1,1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.Conv2D(256, (16, 16), strides=(1,1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
model.add(layers.MaxPooling2D((3, 3), strides=(2,2), padding='same'))
model.add(layers.Flatten())
model.add(layers.Dense(num_classes, kernel_initializer='glorot_uniform',activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_data_training, y_data_training, epochs=100, batch_size=90)
model.save('save_path')

model.evaluate(x_data_test, y_data_test)